In [1]:
### --- Standard libraries ---
import pandas as pd
import numpy as np
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import sys

### --- Genomic tools ---
import bioframe
import pysam
from pyfaidx import Fasta  # You'll use this later for sequence extraction

### --- Custom modules ---
#from dna_io import dna_1hot
#import models as mod
import ChipFold.utils.convert as conv
import ChipFold.utils.experimentalpath as path
from ChipFold.ml.funcs import dna_1hot
import ChipFold.ml.funcs as func

In [2]:
ctcfbeds = path.ctcf_peaks 
ctcfmotif = path.ctcf_motif

In [24]:
ctcf = bioframe.read_table(ctcfbeds, schema='bed')
ctcf['mid']=(ctcf.end+ctcf.start)/2
motif=bioframe.read_table(path.ctcf_motifs)
motif=motif.rename(columns={0: 'chrom', 1: 'start',2:'end',3:'name',4:'score',5:'pval',6:'strand'})
#df = pd.read_csv(path.ctcf_peaks, sep="\t")
peaksmotifs = bioframe.overlap(ctcf,motif,how='inner')[['chrom','start','end','score','strand_']]
peaksmotifs = peaksmotifs.rename(columns={"strand_": "strand"})
peaksmotifs[44:48]

,chrom,start,end,score,strand
44,chr4,7891164,7891491,40.44050,+
45,chr4,7891164,7891491,40.44050,+
46,chr4,7891164,7891491,40.44050,+
47,chr4,7987876,7988126,12.40094,+


In [27]:
conv.convert_ctcf_occupancy(peaksmotifs,ctcfbeds)


KeyError: "['mid'] not in index"